In [2]:
import os
import platform
import sys
import dotenv
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import UnstructuredHTMLLoader
DOT_ENV_PATH = "/Users/danielgeorge/Documents/work/ml/hypolab/Synapse/server/node/.env"
dotenv.load_dotenv(DOT_ENV_PATH)

True

In [3]:
# Read all the lines from snapshots.txt
snapshot_paths = []
with open("snapshots", "r") as snapshot_file:
    lines = snapshot_file.readlines()
    for line in lines:
        snapshot_paths.append(line.strip())

In [4]:
snapshot_paths[0:10]

['/Users/danielgeorge/Zotero/storage/HI6BG4VW/messing-around-with-word2vec.html',
 '/Users/danielgeorge/Zotero/storage/8RKG8IEZ/mrbeast.html',
 '/Users/danielgeorge/Zotero/storage/GXAQX2CC/learn-in-public.html',
 '/Users/danielgeorge/Zotero/storage/FL58XPY8/poxJu.html',
 '/Users/danielgeorge/Zotero/storage/UT4VHUXY/methods-of-prompt-programming.html',
 '/Users/danielgeorge/Zotero/storage/9Y5YLT3P/einstein-productive-thought-combinatory-creativity.html',
 '/Users/danielgeorge/Zotero/storage/MGQIXPC2/372299.html',
 '/Users/danielgeorge/Zotero/storage/9TUU9RY6/ch01s06.html',
 '/Users/danielgeorge/Zotero/storage/7IBZYBUB/mimesis.html',
 '/Users/danielgeorge/Zotero/storage/AZQU42SC/a-technique-for-producing-ideas.html']

In [5]:
from langchain.document_loaders import BSHTMLLoader

In [8]:
documents = []
print(f"Processing {len(snapshot_paths)} snapshots")
for snapshot_path in snapshot_paths:
    # Load the document
    document = BSHTMLLoader(snapshot_path).load()
    # Add the document to the list of documents
    documents += document
    print(snapshot_path)
    # Splitting the text into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    texts = text_splitter.split_documents(documents)
    # Here we are using OpenAI embeddings but in future we will swap out to local embeddings
    embedding = OpenAIEmbeddings()
    documents = []
    # Check if the path is of a real directory
    db_path = 'db' 
    if not os.path.isdir(db_path):
        # Create the directory
        os.mkdir(db_path)
        print("Creating database")
        # Create the database
        vectordb = Chroma.from_documents(documents=texts,
                                     embedding=embedding,
                                     persist_directory=db_path)
    else:
        # Load the database
        vectordb = Chroma(persist_directory=db_path, embedding_function=embedding)
        vectordb.add_documents(texts)

Processing 76 snapshots
/Users/danielgeorge/Zotero/storage/HI6BG4VW/messing-around-with-word2vec.html
/Users/danielgeorge/Zotero/storage/8RKG8IEZ/mrbeast.html
/Users/danielgeorge/Zotero/storage/GXAQX2CC/learn-in-public.html
/Users/danielgeorge/Zotero/storage/FL58XPY8/poxJu.html
/Users/danielgeorge/Zotero/storage/UT4VHUXY/methods-of-prompt-programming.html
/Users/danielgeorge/Zotero/storage/9Y5YLT3P/einstein-productive-thought-combinatory-creativity.html
/Users/danielgeorge/Zotero/storage/MGQIXPC2/372299.html
/Users/danielgeorge/Zotero/storage/9TUU9RY6/ch01s06.html
/Users/danielgeorge/Zotero/storage/7IBZYBUB/mimesis.html
/Users/danielgeorge/Zotero/storage/AZQU42SC/a-technique-for-producing-ideas.html
/Users/danielgeorge/Zotero/storage/9JYIKNLW/monocle.html
/Users/danielgeorge/Zotero/storage/MGL8VX5N/text-universal-interface.html
/Users/danielgeorge/Zotero/storage/W9Y9NS4M/1803.html
/Users/danielgeorge/Zotero/storage/5NILPXND/the-waluigi-effect-mega-post.html
/Users/danielgeorge/Zotero/s